<a href="https://colab.research.google.com/github/zelal-Eizaldeen/DLH_Project_HurtfulWords_Paper/blob/main/Table4_Fairness_metric_recreation_using_preatrained_model_pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#Import modules
import sys
sys.path.append('/content/drive/MyDrive/Payel-DLH-related/Colab Notebooks')
import pandas as pd
from pathlib import Path
import os
from transformers import BertForMaskedLM, BertModel, BertTokenizer,AutoModelForMaskedLM, AutoTokenizer
import torch
import pandas as pd

In [3]:
SCIBERT_DIR = Path('/content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels')

In [4]:
baseline_model_path = SCIBERT_DIR / 'Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512'

In [5]:
print(baseline_model_path)

/content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels/Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512


In [6]:
# DEMOG_FILE = Path('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files')
DEMOG_FILE = Path('/content/drive/MyDrive/Payel-DLH-related/DataFiles/cleaned_data/data.csv')
MIMIC_FILE = Path('/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data/phenotyping/train')
MIMIC_FILE_TEST = Path('/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data/phenotyping/test')

In [7]:
df_demog = pd.read_csv(DEMOG_FILE)

In [8]:
print(len(df_demog))

5018


In [9]:
df_demog.columns

Index(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input'],
      dtype='object')

In [10]:
# df_demog=df_demog.reset_index()()
df_demog=df_demog.rename(columns={'row_id':'note_id'})
df_demog['bert_input'].head()

,bert_input
0,"Patient made CMO last night, bradycardic 20-30..."
1,"Patient made CMO last night, bradycardic 20-30..."
2,PHIAGEPHI yr old female admitted to the micu s...
3,PHIAGEPHI yr old female admitted to the micu s...
4,"Comfort care (CMO, Comfort Measures) Assess..."


In [11]:
df_demog['bert_input'][1]

'Patient made CMO last night, bradycardic 20-30 bpm when I came in,    apneic, appears comfortable. Family at bedside with patient, emotional    support given. Went into asystole, house officer intormed. Pronounced    dead by doctor PHINAMEPHI at  Family offered for autopsy which they    refused to be done to the patient. Post mortem care done, after family    have spent time with patient. Patient s wife not at bedside, children    will inform their mother personally. PHINAMEPHI rites  sacrament of sick    done yesterday. Patient was brought to PHINAMEPHI with name band intact. Comfort care (CMO, Comfort Measures)    Assessment:    Throughout shift pt. became increasingly hypotensive despite being    given blood and albumin. Sats progressively worsened on 100% FIO    Family called and MDs conformed CMO status. Action:    PRN morphine as needed. Morphine gtt ordered but not hung as pt. unresponsive and appears comfortable. Emotional support to family. Response:    Pt. appears comfortabl

In [12]:
df_mimic = pd.read_csv(MIMIC_FILE/'listfile.csv',index_col=0)
df_mimic = df_mimic.reset_index()

In [13]:
df_mimic.head(5)

,stay,period_length,Acute and unspecified renal failure,Acute cerebrovascular disease,Acute myocardial infarction,Cardiac dysrhythmias,Chronic kidney disease,Chronic obstructive pulmonary disease and bronchiectasis,Complications of surgical procedures or medical care,Conduction disorders,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,10036_episode1_timeseries.csv,43.3632,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,1004_episode1_timeseries.csv,144.9864,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,1004_episode2_timeseries.csv,53.8656,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,10064_episode1_timeseries.csv,4.9680,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10093_episode1_timeseries.csv,21.6408,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [14]:

df_mimic['subject_id'] = df_mimic['stay'].str.split('_').str[0]

In [15]:
df_mimic_test = pd.read_csv(MIMIC_FILE_TEST/'listfile.csv',index_col=0)
df_mimic_test = df_mimic_test.reset_index()

df_mimic_test['subject_id'] = df_mimic_test['stay'].str.split('_').str[0]

In [16]:
print(len(df_mimic_test))
print(len(df_mimic))

225
1261


In [16]:
df_demog[['language','language_to_use']].head(100)

,language,language_to_use
0,ENGL,English
1,ENGL,English
2,ENGL,English
3,ENGL,English
4,ENGL,English
...,...,...
95,ENGL,English
96,ENGL,English
97,ENGL,English
98,ENGL,English


In [ ]:
# # df_demog['subject_id_x'] = df_demog['subject_id_x'].astype(int)
# # # df_demog = df_demog[['subject_id_x','gender','ethnicity','language_to_use','insurance']]
# # df_mimic['subject_id'] = df_mimic['subject_id'].astype(int)
# merged_df1 = pd.merge(df_demog, df_mimic, on='subject_id', how='inner')
# merged_df1.head()

,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,367228,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 05:18:15,Physician,Intensivist Note,19882.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,367228,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 05:18:15,Physician,Intensivist Note,19882.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_demog.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input'],
      dtype='object')

In [18]:
# df_demog=df_demog.rename(columns={'subject_id_x':'subject_id'})
df_demog['subject_id'] = df_demog['subject_id'].astype(int)
# df_demog = df_demog[['subject_id_x','gender','ethnicity','language_to_use','insurance']]
df_mimic['subject_id'] = df_mimic['subject_id'].astype(int)
df_mimic_test['subject_id'] = df_mimic_test['subject_id'].astype(int)
merged_df = pd.merge(df_demog, df_mimic, on='subject_id', how='inner')
merged_df.head()


,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,316599,29080,181664.0,2163-02-23,2163-02-23 09:11:00,2163-02-23 09:11:59,Nursing,Nursing Progress Note,21297.0,NaN,...,1,1,0,0,0,0,1,0,0,0
1,316573,29080,181664.0,2163-02-23,2163-02-23 04:31:00,2163-02-23 04:31:07,Nursing,Nursing Progress Note,20706.0,NaN,...,1,1,0,0,0,0,1,0,0,0
2,315769,29548,182189.0,2195-01-18,2195-01-18 16:24:00,2195-01-18 16:25:03,Physician,Physician Resident Admission Note,15235.0,NaN,...,0,1,0,1,0,0,0,1,0,0
3,315777,29548,182189.0,2195-01-18,2195-01-18 16:50:00,2195-01-18 16:50:40,Physician,Physician Attending Admission Note,20047.0,NaN,...,0,1,0,1,0,0,0,1,0,0
4,317118,30699,108670.0,2121-01-07,2121-01-07 05:50:00,2121-01-07 05:50:42,Nursing,Nursing Progress Note,21290.0,NaN,...,0,0,0,0,0,0,0,1,0,0


In [19]:
merged_df_test = pd.merge(df_demog, df_mimic_test, on='subject_id', how='inner')
merged_df_test.head()

,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,325855,28313,156864.0,2117-04-04,2117-04-04 01:30:00,2117-04-04 05:23:20,Nursing,Nursing Progress Note,18469.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,325855,28313,156864.0,2117-04-04,2117-04-04 01:30:00,2117-04-04 05:23:20,Nursing,Nursing Progress Note,18469.0,NaN,...,1,0,0,0,0,0,0,1,1,1
2,325844,28313,156864.0,2117-04-04,2117-04-04 01:30:00,2117-04-04 01:30:36,Nursing,Nursing Progress Note,18469.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,325844,28313,156864.0,2117-04-04,2117-04-04 01:30:00,2117-04-04 01:30:36,Nursing,Nursing Progress Note,18469.0,NaN,...,1,0,0,0,0,0,0,1,1,1
4,325847,28313,156864.0,2117-04-04,2117-04-04 03:15:00,2117-04-04 03:15:16,Physician,Physician Resident Admission Note,18767.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [20]:
print(len(merged_df_test))
print(len(merged_df))

945
4490


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/phenotype_and_demog.csv',index=False)

In [ ]:
merged_df_test.to_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/phenotype_and_demog_test.csv',index=False)

Now that the demographics file (notes) and the phenotypes (labels) have been merges, let's do some more cleaning:

Split each note into sentences, then aggregate short sentences into chunks of at least 20 tokens.

In [21]:
!pip install nltk

In [22]:
import nltk
nltk.data.path.append('/root/nltk_data')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [23]:
nltk.data.path.append('/root/nltk_data')

In [24]:
import os
print(os.listdir('/root/nltk_data/tokenizers'))

['punkt', 'punkt_tab.zip', 'punkt.zip', 'punkt_tab']


In [ ]:
# merged_df = pd.read_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/phenotype_and_demog.csv')
# merged_df.head(1)

,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [25]:
merged_df_copy=merged_df

In [26]:
merged_df_test_copy=merged_df_test

In [27]:
print(len(merged_df))

4490


In [28]:
print(len(merged_df_test))

945


In [29]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

Phase 1 tokenization:

Phase 2 tokenization of same train dataset

Phase 2 tokenization of test dataset

In [30]:
from transformers import BertTokenizer
from tqdm import tqdm
import torch

# Setup
tqdm.pandas()
tokenizer = BertTokenizer.from_pretrained(baseline_model_path)

# Sliding Window Tokenization Function
def sliding_window_tokenize(text, tokenizer, max_len=512, stride=64):
    encoded = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_len,
        stride=stride,
        return_overflowing_tokens=True,
        return_tensors='pt'
    )
    return {
        "input_ids": encoded["input_ids"],              # Shape: [num_chunks, max_len]
        "attention_mask": encoded["attention_mask"]
    }

# Apply per note (from cleaned sentences)
def process_note_with_sliding_window(row, tokenizer, max_len=512, stride=64):
    note = " ".join(row['cleaned_sent'])  # Join all cleaned sentences into one string
    return sliding_window_tokenize(note, tokenizer, max_len=max_len, stride=stride)

# Apply to the DataFrame
merged_df_copy['tokenized_notes'] = merged_df_copy.progress_apply(
    lambda row: process_note_with_sliding_window(row, tokenizer, max_len=512, stride=64),
    axis=1
)


100%|██████████| 4490/4490 [08:17<00:00,  9.03it/s]


In [31]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

Creating label columns

In [32]:
label_col = ['Acute and unspecified renal failure',
       'Acute cerebrovascular disease', 'Acute myocardial infarction',
       'Cardiac dysrhythmias', 'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Complications of surgical procedures or medical care',
       'Conduction disorders', 'Congestive heart failure; nonhypertensive',
       'Coronary atherosclerosis and other heart disease',
       'Diabetes mellitus with complications',
       'Diabetes mellitus without complication',
       'Disorders of lipid metabolism', 'Essential hypertension',
       'Fluid and electrolyte disorders', 'Gastrointestinal hemorrhage',
       'Hypertension with complications and secondary hypertension',
       'Other liver diseases', 'Other lower respiratory disease',
       'Other upper respiratory disease',
       'Pleurisy; pneumothorax; pulmonary collapse',
       'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
       'Respiratory failure; insufficiency; arrest (adult)',
       'Septicemia (except in labor)', 'Shock']

In [33]:
merged_df_copy['labels'] = merged_df_copy[label_col].values.tolist()

In [34]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

In [35]:
merged_df_copy['labels'][0]

[1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0]

Phase 1 training prep

Phase 2 training prep

In [36]:
from torch.utils.data import Dataset
import torch

class ClinicalNotesDataset(Dataset):
    def __init__(self, df, max_len=512):
        self.samples = []

        for _, row in df.iterrows():
            # Get tokenized notes and labels
            tokenized = row['tokenized_notes']
            label = torch.tensor(row['labels'], dtype=torch.float)  # Assuming it's a multi-label tensor

            # Extract chunks for each tokenized note
            input_ids_chunks = tokenized['input_ids']  # Shape: [num_chunks, max_len]
            attention_mask_chunks = tokenized['attention_mask']

            # Create a sample for each chunk of the note
            for input_ids, attention_mask in zip(input_ids_chunks, attention_mask_chunks):
                # Repeat the label for every chunk (because label applies to the entire note)
                self.samples.append({
                    'input_ids': input_ids,
                    'attention_mask': attention_mask,
                    'labels': label
                })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return {
            'input_ids': self.samples[idx]['input_ids'],
            'attention_mask': self.samples[idx]['attention_mask'],
            'labels': self.samples[idx]['labels']
        }


Phase 1

Phase 2

In [37]:
dataset = ClinicalNotesDataset(merged_df_copy)
sample = dataset[0]

print(sample['input_ids'].shape)       # torch.Size([512])
print(sample['attention_mask'].shape)  # torch.Size([512])
print(sample['labels'].shape)          # torch.Size([25])

torch.Size([512])
torch.Size([512])
torch.Size([25])


In [38]:
train_dataset_phase2 = ClinicalNotesDataset(merged_df_copy, max_len=512)

Phase 1 model training:

Phase 2 model training and evaluating

In [39]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score
import numpy as np

# Load the pretrained Phase 1 model
model = BertForSequenceClassification.from_pretrained(baseline_model_path,num_labels=25)

# Define training arguments for evaluation
eval_args = TrainingArguments(
    output_dir="./eval_results",  # You can update this path
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    do_train=False,
    do_eval=True
)

# Custom metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)

    recall = recall_score(labels, preds, average='micro')
    precision = precision_score(labels, preds, average='micro')
    f1 = f1_score(labels, preds, average='micro')
    accuracy = accuracy_score(labels, preds)

    return {
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'accuracy': accuracy,
    }

# Initialize Trainer for evaluation
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=train_dataset_phase2,
    compute_metrics=compute_metrics
)

# Run evaluation
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# Optional: Get predictions
predictions = trainer.predict(train_dataset_phase2)
y_true = predictions.label_ids
y_pred = (predictions.predictions > 0.5).astype(int)

# Compute specificity
specificities = []
for i in range(y_true.shape[1]):
    cm = confusion_matrix(y_true[:, i], y_pred[:, i])
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificities.append(specificity)
    else:
        specificities.append(np.nan)

average_specificity = np.nanmean(specificities)
print(f"Average Specificity: {average_specificity:.2f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels/Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cpayel25 (cpayel25-university-of-illinois-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Evaluation Results: {'eval_loss': 0.7113735675811768, 'eval_model_preparation_time': 0.0049, 'eval_recall': 0.04309809143364403, 'eval_precision': 0.2111328549684714, 'eval_f1': 0.07158391389288216, 'eval_accuracy': 0.005790645879732739, 'eval_runtime': 123.3315, 'eval_samples_per_second': 36.406, 'eval_steps_per_second': 2.278}
Average Specificity: 0.96


In [40]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

Table 4 structure formation:

In [41]:
gender_map = {
    'F': 'Female',
    'M': 'Male'
}

merged_df_copy['gender_mapped'] = merged_df_copy['gender'].map(gender_map)
ethnicity_map = {
    "WHITE": "White",
    "BLACK": "Black",
    "ASIAN": "Asian",
    "HISPANIC/LATINO": "Hispanic",
    "OTHER": "Other",
    "UNKNOWN/NOT SPECIFIED": "Other"
}

merged_df_copy['ethnicity_mapped'] = merged_df_copy['ethnicity_to_use'].map(ethnicity_map)


language_map = {
    "English": "English",
    "Other": "Other",
    "Missing": "Other"  # You can also exclude Missing instead
}

merged_df_copy['language_mapped'] = merged_df_copy['language_to_use'].map(language_map)
insurance_map = {
    "Medicare": "Medicare",
    "Private": "Private",
    "Medicaid": "Medicaid",
    "Government": "Other",
    "Self Pay": "Other",
    "Other": "Other"
}

merged_df_copy['insurance_mapped'] = merged_df_copy['insurance'].map(insurance_map)


In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix
from tqdm import tqdm
from IPython.display import display  # For rendering the table

# --- Config ---
demographic_columns = {
    "gender_mapped": [("Male", "Female")],  # Ensuring Male vs Female
    "language_mapped": [("English", "Other")],
    "ethnicity_mapped": [("White", "Other"), ("Black", "Other"), ("Hispanic", "Other"), ("Asian", "Other"), ("Other", "Other")],  # Added "Other" vs "Other" here
    "insurance_mapped": [("Medicare", "Other"), ("Private", "Other"), ("Medicaid", "Other")]
}
n_bootstrap = 100
alpha = 0.05

# --- Results placeholder ---
table4_results = []

# Efficient specificity function
def safe_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    if cm.shape == (2, 2):
        tn, fp, _, _ = cm.ravel()
        denom = tn + fp
        return tn / denom if denom else 0.0
    return 0.0

# Safe recall
def safe_recall(y_true, y_pred):
    return recall_score(y_true, y_pred) if len(np.unique(y_true)) > 1 else 0.0

# Bootstrap difference with numpy vectorization
def bootstrap_metric_diff(y_true1, y_pred1, y_true2, y_pred2, metric_fn):
    n1, n2 = len(y_true1), len(y_true2)
    if n1 == 0 or n2 == 0:
        return 0.0, False, False
    diffs = []
    for _ in range(n_bootstrap):
        idx1 = np.random.randint(0, n1, n1)
        idx2 = np.random.randint(0, n2, n2)
        try:
            m1 = metric_fn(y_true1[idx1], y_pred1[idx1])
            m2 = metric_fn(y_true2[idx2], y_pred2[idx2])
            diffs.append(m1 - m2)
        except:
            continue
    if not diffs:
        return 0.0, False, False
    diffs = np.array(diffs)
    ci_low, ci_high = np.percentile(diffs, [100 * alpha / 2, 100 * (1 - alpha / 2)])
    significant = (ci_low > 0) or (ci_high < 0)
    favors_group1 = diffs.mean() > 0
    return diffs.mean(), significant, favors_group1

# Main loop with tqdm
for fairness_attr, group_pairs in demographic_columns.items():
    for group1, group2 in group_pairs:
        recall_sig = parity_sig = spec_sig = 0
        recall_fav = parity_fav = spec_fav = 0

        # Precompute masks
        if group1 == "Other":
            # When group1 is the literal string "Other", match only rows labeled "Other"
            mask1 = (merged_df_copy[fairness_attr] == "Other").values
        else:
            mask1 = (merged_df_copy[fairness_attr] == group1).values

        if group2 == "Other":
            # When group2 is "Other", dynamically create mask2 as all rows NOT in group1
            mask2 = (merged_df_copy[fairness_attr] != group1).values
        else:
            mask2 = (merged_df_copy[fairness_attr] == group2).values

        for label_idx in tqdm(range(y_true.shape[1]), desc=f"{fairness_attr} {group1} vs {group2}"):
            y_true1, y_pred1 = y_true[mask1, label_idx], y_pred[mask1, label_idx]
            y_true2, y_pred2 = y_true[mask2, label_idx], y_pred[mask2, label_idx]

            if len(y_true1) < 1 or len(y_true2) < 1:
                continue

            # Bootstrapping
            _, sig_r, fav_r = bootstrap_metric_diff(y_true1, y_pred1, y_true2, y_pred2, safe_recall)
            _, sig_p, fav_p = bootstrap_metric_diff(y_true1, y_pred1, y_true2, y_pred2, accuracy_score)
            _, sig_s, fav_s = bootstrap_metric_diff(y_true1, y_pred1, y_true2, y_pred2, safe_specificity)

            recall_sig += sig_r
            parity_sig += sig_p
            spec_sig += sig_s

            recall_fav += sig_r and fav_r
            parity_fav += sig_p and fav_p
            spec_fav += sig_s and fav_s

        table4_results.append({
            "Fairness Definition": fairness_attr,
            "Comparison (Group 1 vs Group 2)": f"{group1} vs {group2}",
            "Recall Gap": f"{recall_sig} ({recall_fav / max(recall_sig, 1) * 100:.1f}%)",
            "Parity Gap": f"{parity_sig} ({parity_fav / max(parity_sig, 1) * 100:.1f}%)",
            "Specificity Gap": f"{spec_sig} ({spec_fav / max(spec_sig, 1) * 100:.1f}%)"
        })

# Final output
# Convert to DataFrame
table4_df = pd.DataFrame(table4_results)

# Add a MultiIndex header above the Gap columns
table4_df.columns = pd.MultiIndex.from_tuples([
    ("Fairness Definition", ""),
    ("Comparison (Group 1 vs Group 2)", ""),
    ("Significant Differences by Fairness Definition", "Recall Gap"),
    ("Significant Differences by Fairness Definition", "Parity Gap"),
    ("Significant Differences by Fairness Definition", "Specificity Gap")
])

# Style the DataFrame
styled_table = (
    table4_df.style
    .set_table_styles(
        [   {'selector': 'th', 'props': [('font-weight', 'bold'), ('border', '1px solid black')]},
            {'selector': 'td', 'props': [('border', '1px solid black')]},
            {'selector': 'table', 'props': [('border-collapse', 'collapse')]},
        ]
    )
    .apply(lambda x: ['font-weight: bold' if x.name == 'Fairness Definition' else '' for _ in x], axis=1)
    .set_properties(**{'text-align': 'center'})
    .set_caption("Final Fairness Table 4 (Styled)")
)

# Display in Jupyter/IPython
display(styled_table)

# Save as HTML
styled_table.to_html("final_fairness_table4.html", escape=False, index=False)


insurance_mapped Medicaid vs Other: 100%|██████████| 25/25 [00:15<00:00,  1.63it/s]


**Some explanation on the Comparison:**
English vs. Other
→ You're comparing model performance between patients who speak English and those who speak other languages.

Recall Gap:
2 (50.0%)
→ Out of the 25 clinical labels:

2 of them had a statistically significant difference in recall (sensitivity) between English and Other.

Of those 2, 50% of the time (so, 1 label), English performed better (i.e., had higher recall).

Parity Gap:
20 (25.0%)
→ 20 labels had a significant difference in accuracy (aka parity).

25% of those (so, 5 labels) favored English over Other in accuracy.

Specificity Gap:
4 (25.0%)
→ 4 labels had a significant difference in specificity.

25% of those (1 label) favored English over Other in specificity.



In [1]:
print(merged_df_copy["gender_mapped"].value_counts())


NameError: name 'merged_df_copy' is not defined

## Just checks, will be deleted later

In [ ]:
import pandas as pd
import numpy as np

# Assuming y_true and y_pred are NumPy arrays from the predictions
# Let's say you want to inspect the first 5 subjects

indices_to_inspect = [0, 1, 2, 3, 4]  # Example indices of the subjects to check

# Extracting the true and predicted values for those indices
y_true_subset = y_true[indices_to_inspect]
y_pred_subset = y_pred[indices_to_inspect]

# Extract the corresponding subjects from merged_df_test_copy
subjects_subset = merged_df_copy.iloc[indices_to_inspect]

# Combine them into a DataFrame for easier inspection
results_df = subjects_subset.copy()
results_df['y_true'] = [y_true_subset[i] for i in range(len(y_true_subset))]
results_df['y_pred'] = [y_pred_subset[i] for i in range(len(y_pred_subset))]

# Display the results
print(results_df)


   note_id  subject_id   hadm_id   chartdate            charttime  \
0   316599       29080  181664.0  2163-02-23  2163-02-23 09:11:00   
1   316573       29080  181664.0  2163-02-23  2163-02-23 04:31:00   
2   315769       29548  182189.0  2195-01-18  2195-01-18 16:24:00   
3   315777       29548  182189.0  2195-01-18  2195-01-18 16:50:00   
4   317118       30699  108670.0  2121-01-07  2121-01-07 05:50:00   

             storetime   category                         description  \
0  2163-02-23 09:11:59    Nursing               Nursing Progress Note   
1  2163-02-23 04:31:07    Nursing               Nursing Progress Note   
2  2195-01-18 16:25:03  Physician   Physician Resident Admission Note   
3  2195-01-18 16:50:40  Physician  Physician Attending Admission Note   
4  2121-01-07 05:50:42    Nursing               Nursing Progress Note   

      cgid  iserror  ... Other upper respiratory disease  \
0  21297.0      NaN  ...                               0   
1  20706.0      NaN  ...  

In [ ]:
print(merged_df_copy['gender'].value_counts())
print(merged_df_copy['ethnicity_to_use'].value_counts())
print(merged_df_copy['language_to_use'].value_counts())

gender
F    2433
M    2057
Name: count, dtype: int64
ethnicity_to_use
WHITE                    3264
BLACK                     594
UNKNOWN/NOT SPECIFIED     420
ASIAN                      99
OTHER                      67
HISPANIC/LATINO            46
Name: count, dtype: int64
language_to_use
English    2341
Missing    1835
Other       314
Name: count, dtype: int64


In [ ]:
print(merged_df_copy['gender'].value_counts(dropna=False))
print(merged_df_copy['ethnicity_to_use'].value_counts(dropna=False))

gender
NaN    4490
Name: count, dtype: int64
ethnicity_to_use
NaN    4490
Name: count, dtype: int64


In [ ]:
print(model.config)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,
  

Extra:










After model 2 finetuning:
 [16/16 12:16, Epoch 1/1]
Epoch	Training Loss	Validation Loss	Recall	Precision	F1	Accuracy
1	No log	0.665566	0.030457	0.083333	0.044610	0.000000
{'eval_loss': 0.6655661463737488, 'eval_recall': 0.030456852791878174, 'eval_precision': 0.08333333333333333, 'eval_f1': 0.04460966542750929, 'eval_accuracy': 0.0, 'eval_runtime': 94.6844, 'eval_samples_per_second': 0.549, 'eval_steps_per_second': 0.042, 'epoch': 1.0}
Test Set Accuracy: 0.0
Test Set F1 Score: 0.04460966542750929
Test Set Recall: 0.030456852791878174
Average Specificity: 0.94
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(




That output isn’t necessarily a code failure, but it does signal a model performance failure — the model barely learned anything useful. Let’s break down what happened and what the messages mean:

🛑 What Went Wrong
🔢 Metrics are very low:
Accuracy: 0.0 (your model didn't get any label combination correct).

Recall: 0.03

Precision: 0.08

F1: 0.04

These are very close to random guessing — so yes, the model’s performance is essentially a failure.

🧠 Likely Causes
Frozen BERT backbone

You're only training the classifier head (you froze all model.bert.parameters()).

If Phase 1 fine-tuning didn't give you good domain-specific representations, freezing BERT in Phase 2 might leave the classifier powerless to adapt to new patterns.

✅ Fix: Try unfreezing some or all of the BERT layers for Phase 2 and fine-tune the whole model.

Too little training (1 epoch)

One epoch might not be enough, especially with a small learning head and a frozen encoder.

✅ Fix: Try more epochs (e.g., 3–5) and monitor validation F1 after each.

Bad threshold

Threshold 0.5 might be too strict — possibly resulting in very few positive predictions.

✅ Fix: Try threshold tuning (e.g., test thresholds from 0.1 to 0.9 and pick the best based on validation F1).

Label imbalance

If some labels are rare, metrics (especially F1/recall) can be skewed badly.

✅ Fix: Consider:

Oversampling underrepresented labels.

Using a weighted loss like BCEWithLogitsLoss(pos_weight=...).

⚠️ About the Warning:
plaintext
Copy
Edit
UserWarning: A single label was found in 'y_true' and 'y_pred'.
This means that for some labels, either the model predicted only 0s (no positives), or your test labels had only 0s. As a result, the confusion matrix had only one class — which confuses confusion_matrix().

✅ Fix: When computing confusion matrices in multi-label setups, pass all possible labels explicitly:

python
Copy
Edit
confusion_matrix(y_true[:, i], y_pred[:, i], labels=[0, 1])
✅ Summary
This isn’t a bug, but it's a sign the Phase 2 training isn’t effective yet. To improve:

Try unfreezing the BERT layers.

Train longer.

Tune the threshold.

Handle label imbalance better.

Make sure your datasets are clean and correctly labeled.

Want help modifying the code to unfreeze BERT or run threshold tuning?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, multilabel_confusion_matrix

# Function to calculate fairness gaps for a binary subgroup vs. the rest (reference group)
def calculate_fairness_vs_others(df, subgroup_col, reference_group, y_true, y_pred):
    # Binary comparison: ref vs all others
    mask_ref = df[subgroup_col] == reference_group
    mask_other = df[subgroup_col] != reference_group

    idx1 = df[mask_ref].index
    idx2 = df[mask_other].index

    if len(idx1) == 0 or len(idx2) == 0:
        print(f"[DEBUG] Skipping {subgroup_col} - not enough samples for {reference_group} vs. others")
        print(f"[DEBUG] {reference_group} count = {len(idx1)}, Other count = {len(idx2)}")

        # Optional: peek at label distribution per group
        print(f"[DEBUG] Label distribution for {reference_group}:")
        print(df.loc[idx1, 'labels'].value_counts())

        print(f"[DEBUG] Label distribution for Others:")
        print(df.loc[idx2, 'labels'].value_counts())

        return {
            "Fairness Definition": subgroup_col,
            "Comparison (Group 1 vs. Group 2)": f"{reference_group} vs. Other",
            "Recall Gap": np.nan,
            "Parity Gap": np.nan,
            "Specificity Gap": np.nan
        }

    g1_recall, g2_recall = [], []
    g1_parity, g2_parity = [], []
    g1_specificity, g2_specificity = [], []

    # Calculate metrics for each task (column)
    for i in range(y_true.shape[1]):
        r1 = recall_score(y_true[idx1, i], y_pred[idx1, i], zero_division=0)
        r2 = recall_score(y_true[idx2, i], y_pred[idx2, i], zero_division=0)
        g1_recall.append(r1)
        g2_recall.append(r2)

        p1 = y_pred[idx1, i].mean()
        p2 = y_pred[idx2, i].mean()
        g1_parity.append(p1)
        g2_parity.append(p2)

        cm1 = multilabel_confusion_matrix(y_true[idx1, i], y_pred[idx1, i])[0]
        cm2 = multilabel_confusion_matrix(y_true[idx2, i], y_pred[idx2, i])[0]
        tn1, fp1 = cm1[0, 0], cm1[0, 1]
        tn2, fp2 = cm2[0, 0], cm2[0, 1]
        s1 = tn1 / (tn1 + fp1) if (tn1 + fp1) > 0 else 0
        s2 = tn2 / (tn2 + fp2) if (tn2 + fp2) > 0 else 0
        g1_specificity.append(s1)
        g2_specificity.append(s2)

    # Gaps and % favoring reference group
    recall_diffs = np.array(g1_recall) - np.array(g2_recall)
    parity_diffs = np.array(g1_parity) - np.array(g2_parity)
    specificity_diffs = np.array(g1_specificity) - np.array(g2_specificity)

    # Convert to percentage
    recall_gap_pct = round(np.mean(np.abs(recall_diffs)) * 100, 2)
    parity_gap_pct = round(np.mean(np.abs(parity_diffs)) * 100, 2)
    specificity_gap_pct = round(np.mean(np.abs(specificity_diffs)) * 100, 2)

    # Absolute gap and percentage gap formatting
    recall_gap_str = f"{round(np.mean(np.abs(recall_diffs)), 2)} ({recall_gap_pct}%)"
    parity_gap_str = f"{round(np.mean(np.abs(parity_diffs)), 2)} ({parity_gap_pct}%)"
    specificity_gap_str = f"{round(np.mean(np.abs(specificity_diffs)), 2)} ({specificity_gap_pct}%)"

    return {
        "Fairness Definition": subgroup_col,
        "Comparison (Group 1 vs. Group 2)": f"{reference_group} vs. Other",
        "Recall Gap": recall_gap_str,
        "Parity Gap": parity_gap_str,
        "Specificity Gap": specificity_gap_str
    }

# Main block to calculate fairness gaps for Gender, Language, Ethnicity, and Insurance
results = []

# Gender fairness evaluation
results.append(calculate_fairness_vs_others(merged_df_copy, 'gender_mapped', 'Male', y_true, y_pred))

# Language fairness evaluation
results.append(calculate_fairness_vs_others(merged_df_copy, 'language_mapped', 'English', y_true, y_pred))

# Ethnicity fairness evaluation
for group in ['White', 'Black', 'Hispanic', 'Asian', 'Other']:
    results.append(calculate_fairness_vs_others(merged_df_copy, 'ethnicity_mapped', group, y_true, y_pred))

# Insurance fairness evaluation
for group in ['Medicare', 'Private', 'Medicaid']:
    results.append(calculate_fairness_vs_others(merged_df_copy, 'insurance_mapped', group, y_true, y_pred))

# Convert the results to a DataFrame for easier table generation
table4_df = pd.DataFrame(results)

# Check the column names to make sure they are correct
print(table4_df.columns)

# Strip any leading/trailing whitespace from column names if necessary
table4_df.columns = table4_df.columns.str.strip()

# Display the DataFrame as a table in Google Colab
table4_df


Index(['Fairness Definition', 'Comparison (Group 1 vs. Group 2)', 'Recall Gap',
       'Parity Gap', 'Specificity Gap'],
      dtype='object')


,Fairness Definition,Comparison (Group 1 vs. Group 2),Recall Gap,Parity Gap,Specificity Gap
0,gender_mapped,Male vs. Other,0.01 (0.98%),0.0 (0.18%),0.01 (0.98%)
1,language_mapped,English vs. Other,0.0 (0.47%),0.01 (0.64%),0.0 (0.47%)
2,ethnicity_mapped,White vs. Other,0.02 (1.55%),0.01 (0.75%),0.02 (1.55%)
3,ethnicity_mapped,Black vs. Other,0.02 (2.03%),0.01 (0.58%),0.02 (2.03%)
4,ethnicity_mapped,Hispanic vs. Other,0.03 (3.48%),0.01 (0.73%),0.03 (3.48%)
5,ethnicity_mapped,Asian vs. Other,0.1 (9.77%),0.03 (3.06%),0.1 (9.77%)
6,ethnicity_mapped,Other vs. Other,0.01 (1.4%),0.01 (1.04%),0.01 (1.4%)
7,insurance_mapped,Medicare vs. Other,0.01 (0.76%),0.01 (0.66%),0.01 (0.76%)
8,insurance_mapped,Private vs. Other,0.02 (1.52%),0.01 (0.71%),0.02 (1.52%)
9,insurance_mapped,Medicaid vs. Other,0.04 (3.78%),0.01 (1.19%),0.04 (3.78%)
